In [1]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import re
import gensim
from gensim.utils import simple_preprocess
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.optimizers import Adam

## 데이터 로드

In [2]:
# 집
df=pd.read_csv('C:/Users/yeons/OneDrive/바탕 화면/tb_cargomylist_need_columns.csv', encoding='cp949')
df

C:\Users\yeons\AppData\Local\Temp\ipykernel_19256\880000552.py:2: DtypeWarning: Columns (27,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('C:/Users/yeons/OneDrive/바탕 화면/tb_cargomylist_need_columns.csv', encoding='cp949')


,Unnamed: 0,intSeq,intStatus,bShare,intOwMemberSeq,strUpZone,strUpAreaCode,strUpJAddress,strDwZone,strDwAreaCode,...,strDownloadType,intDistance,intDefaultPrice,fltRealWeight,intActApp,intUpDateType,intDwDateType,intCarManageSeq,intMemberSeq,intFeeSend
0,0,92218976,3,1,57037.0,41.0,H050004,경기 광주 도척면,30.0,F010005,...,일반,116,0,7.0,999,1.0,1.0,11017.0,NaN,NaN
1,1,92218977,3,1,57037.0,41.0,H050004,경기 광주 도척면,43.0,J100000,...,일반,113,0,7.0,999,1.0,1.0,11017.0,NaN,NaN
2,2,92218978,3,1,63816.0,41.0,H080017,경기 김포 학운리,41.0,H230304,...,일반,100,0,26.0,2,1.0,1.0,11017.0,12953.0,NaN
3,3,92218979,6,1,55238.0,28.0,D000000,인천,26.0,B000000,...,일반,408,0,27.5,202,1.0,2.0,11017.0,NaN,NaN
4,4,92218980,3,1,41258.0,41.0,H300000,경기 하남,41.0,H060000,...,일반,14,135000,10.0,4,1.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885028,384471,1221530,6,1,38193.0,41.0,H260008,경기 이천 마장면,41.0,H120300,...,지게차,48,0,1.0,93,2.0,2.0,\N,0.0,NaN
1885029,384472,1221531,6,1,38193.0,41.0,H260008,경기 이천 마장면,41.0,H120300,...,지게차,48,0,1.0,93,2.0,2.0,\N,0.0,NaN
1885030,384473,1221532,6,1,38193.0,41.0,H260008,경기 이천 마장면,41.0,H120300,...,지게차,48,0,1.0,93,2.0,2.0,\N,0.0,NaN
1885031,384474,1221533,6,1,53524.0,30.0,F020000,대전 동구,46.0,K080000,...,지게차,231,100000,5.0,2,1.0,2.0,\N,0.0,NaN


In [3]:
# # 학교
# df=pd.read_csv('C:/Users/user/OneDrive - kyonggi.ac.kr/바탕 화면/화물맨프로젝트/화물맨 프로젝트/Ipynb/오토인코더추천시스템/tb_cargomylist_need_columns.csv', encoding='cp949')
# df

In [4]:
# "intOwMemberSeq" 열의 값을 자연수 형태로 변환
df['intOwMemberSeq'] = df['intOwMemberSeq'].fillna(0)
df['intOwMemberSeq'] = df['intOwMemberSeq'].astype(int)

print(df)

         Unnamed: 0    intSeq  intStatus  bShare  intOwMemberSeq  strUpZone  \
0                 0  92218976          3       1           57037       41.0   
1                 1  92218977          3       1           57037       41.0   
2                 2  92218978          3       1           63816       41.0   
3                 3  92218979          6       1           55238       28.0   
4                 4  92218980          3       1           41258       41.0   
...             ...       ...        ...     ...             ...        ...   
1885028      384471   1221530          6       1           38193       41.0   
1885029      384472   1221531          6       1           38193       41.0   
1885030      384473   1221532          6       1           38193       41.0   
1885031      384474   1221533          6       1           53524       30.0   
1885032      384475   1221534          6       1           73964       41.0   

        strUpAreaCode strUpJAddress  strDwZone strD

In [5]:
# 열 삭제 
df = df.drop('Unnamed: 0',axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1885033 entries, 0 to 1885032
Data columns (total 32 columns):
 #   Column           Dtype  
---  ------           -----  
 0   intSeq           int64  
 1   intStatus        int64  
 2   bShare           int64  
 3   intOwMemberSeq   int32  
 4   strUpZone        float64
 5   strUpAreaCode    object 
 6   strUpJAddress    object 
 7   strDwZone        float64
 8   strDwAreaCode    object 
 9   strDwJAddress    object 
 10  strCarType       object 
 11  intCarTypecode   float64
 12  strCarSize       object 
 13  intCarSizecode   float64
 14  intCarUpType     int64  
 15  strPaytype       object 
 16  intPaytypecode   int64  
 17  intTotalPrice    int64  
 18  intMemberPrice   int64  
 19  intArrangePrice  int64  
 20  strGoodsType     object 
 21  strUploadType    object 
 22  strDownloadType  object 
 23  intDistance      int64  
 24  intDefaultPrice  int64  
 25  fltRealWeight    float64
 26  intActApp        object 
 27  intUpDateTyp

## 오토인코더를 사용하기 위해 object 타입 수치형 데이터로 변환

### - labelencoder 사용

In [7]:
encoder = LabelEncoder()
df['strUpAreaCode'] = encoder.fit_transform(df['strUpAreaCode'].astype(str))
df['strUpJAddress'] = encoder.fit_transform(df['strUpJAddress'].astype(str))
df['strDwJAddress'] = encoder.fit_transform(df['strDwJAddress'].astype(str))
df['strUpAreaCode'] = encoder.fit_transform(df['strUpAreaCode'].astype(str))
df['strDwAreaCode'] = encoder.fit_transform(df['strDwAreaCode'].astype(str))
df['strDwJAddress'] = encoder.fit_transform(df['strDwJAddress'].astype(str))
df['strCarType'] = encoder.fit_transform(df['strCarType'].astype(str))
df['strCarSize'] = encoder.fit_transform(df['strCarSize'].astype(str))
df['strPaytype'] = encoder.fit_transform(df['strPaytype'].astype(str))
df['strGoodsType'] = encoder.fit_transform(df['strGoodsType'].astype(str))
df['strUploadType'] = encoder.fit_transform(df['strUploadType'].astype(str))
df['strDownloadType'] = encoder.fit_transform(df['strDownloadType'].astype(str))
df['intActApp'] = encoder.fit_transform(df['intActApp'].astype(str))
df['intCarManageSeq'] = encoder.fit_transform(df['intCarManageSeq'].astype(str))

In [8]:
df

,intSeq,intStatus,bShare,intOwMemberSeq,strUpZone,strUpAreaCode,strUpJAddress,strDwZone,strDwAreaCode,strDwJAddress,...,strDownloadType,intDistance,intDefaultPrice,fltRealWeight,intActApp,intUpDateType,intDwDateType,intCarManageSeq,intMemberSeq,intFeeSend
0,92218976,3,1,57037,41.0,276,2209,30.0,1083,29987,...,1,116,0,7.0,12,1.0,1.0,4,NaN,NaN
1,92218977,3,1,57037,41.0,276,2210,43.0,2660,61100,...,1,113,0,7.0,12,1.0,1.0,4,NaN,NaN
2,92218978,3,1,63816,41.0,339,3384,41.0,1947,3505,...,1,100,0,26.0,2,1.0,1.0,4,12953.0,NaN
3,92218979,6,1,55238,28.0,3995,28850,26.0,379,31401,...,1,408,0,27.5,3,1.0,2.0,4,NaN,NaN
4,92218980,3,1,41258,41.0,1008,11509,41.0,1459,48169,...,1,14,135000,10.0,8,1.0,1.0,9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885028,1221530,6,1,38193,41.0,874,8495,41.0,1612,64693,...,2,48,0,1.0,10,2.0,2.0,8,0.0,NaN
1885029,1221531,6,1,38193,41.0,874,8495,41.0,1612,64693,...,2,48,0,1.0,10,2.0,2.0,8,0.0,NaN
1885030,1221532,6,1,38193,41.0,874,8495,41.0,1612,64693,...,2,48,0,1.0,10,2.0,2.0,8,0.0,NaN
1885031,1221533,6,1,53524,30.0,4351,23374,46.0,2949,45702,...,2,231,100000,5.0,2,1.0,2.0,8,0.0,NaN


## 오토인코더를 사용하기 위해 입력 데이터를 정규화

In [9]:
# 데이터를 정규화합니다.
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df),columns=df.columns)
df_normalized

,intSeq,intStatus,bShare,intOwMemberSeq,strUpZone,strUpAreaCode,strUpJAddress,strDwZone,strDwAreaCode,strDwJAddress,...,strDownloadType,intDistance,intDefaultPrice,fltRealWeight,intActApp,intUpDateType,intDwDateType,intCarManageSeq,intMemberSeq,intFeeSend
0,0.285695,-0.990183,0.034152,0.817190,0.123499,-1.329024,-1.572152,-1.011072,-0.918296,-0.170030,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
1,0.285695,-0.990183,0.034152,0.817190,0.123499,-1.329024,-1.572075,0.448039,0.307204,1.392233,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
2,0.285695,-0.990183,0.034152,1.146550,0.123499,-1.277805,-1.481069,0.223560,-0.246875,-1.499758,...,-0.328421,0.004005,-0.684241,1.178400,-1.758640,-0.389139,-0.908531,-0.784032,-0.307990,NaN
3,0.285695,0.966049,0.034152,0.729785,-1.623514,1.694531,0.493004,-1.460029,-1.465381,-0.099029,...,-0.328421,0.004034,-0.684241,1.356320,-1.416451,-0.389139,0.300922,-0.784032,NaN,NaN
4,0.285695,-0.990183,0.034152,0.050563,0.123499,-0.733906,-0.851235,0.223560,-0.626104,0.742935,...,-0.328421,0.003997,0.147795,-0.719407,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885028,-1.934176,0.966049,0.034152,-0.098350,0.123499,-0.842849,-1.084874,0.223560,-0.507206,1.572647,...,1.613138,0.004000,-0.684241,-1.786924,0.978870,1.647601,0.300922,0.777998,-0.807495,NaN
1885029,-1.934176,0.966049,0.034152,-0.098350,0.123499,-0.842849,-1.084874,0.223560,-0.507206,1.572647,...,1.613138,0.004000,-0.684241,-1.786924,0.978870,1.647601,0.300922,0.777998,-0.807495,NaN
1885030,-1.934176,0.966049,0.034152,-0.098350,0.123499,-0.842849,-1.084874,0.223560,-0.507206,1.572647,...,1.613138,0.004000,-0.684241,-1.786924,0.978870,1.647601,0.300922,0.777998,-0.807495,NaN
1885031,-1.934176,0.966049,0.034152,0.646510,-1.354743,1.983959,0.068515,0.784757,0.531788,0.619060,...,1.613138,0.004017,-0.067918,-1.312472,-1.758640,-0.389139,0.300922,0.777998,-0.807495,NaN


In [10]:
# 1000개만
df1 = df_normalized.iloc[:1000, :]
df1

,intSeq,intStatus,bShare,intOwMemberSeq,strUpZone,strUpAreaCode,strUpJAddress,strDwZone,strDwAreaCode,strDwJAddress,...,strDownloadType,intDistance,intDefaultPrice,fltRealWeight,intActApp,intUpDateType,intDwDateType,intCarManageSeq,intMemberSeq,intFeeSend
0,0.285695,-0.990183,0.034152,0.817190,0.123499,-1.329024,-1.572152,-1.011072,-0.918296,-0.170030,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
1,0.285695,-0.990183,0.034152,0.817190,0.123499,-1.329024,-1.572075,0.448039,0.307204,1.392233,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
2,0.285695,-0.990183,0.034152,1.146550,0.123499,-1.277805,-1.481069,0.223560,-0.246875,-1.499758,...,-0.328421,0.004005,-0.684241,1.178400,-1.758640,-0.389139,-0.908531,-0.784032,-0.30799,NaN
3,0.285695,0.966049,0.034152,0.729785,-1.623514,1.694531,0.493004,-1.460029,-1.465381,-0.099029,...,-0.328421,0.004034,-0.684241,1.356320,-1.416451,-0.389139,0.300922,-0.784032,NaN,NaN
4,0.285695,-0.990183,0.034152,0.050563,0.123499,-0.733906,-0.851235,0.223560,-0.626104,0.742935,...,-0.328421,0.003997,0.147795,-0.719407,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.285720,0.966049,0.034152,0.786727,0.795428,-0.006270,0.801138,0.672517,0.996499,0.966380,...,-0.328421,0.004009,-0.684241,1.297013,-1.758640,-0.389139,0.300922,-0.784032,NaN,NaN
996,0.285720,-0.990183,0.034152,-0.035141,0.123499,-0.759922,-0.933094,0.896996,1.684240,-0.565905,...,-0.328421,0.004015,1.711405,1.059787,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
997,0.285720,0.966049,0.034152,0.657879,0.661042,0.287224,0.938732,0.896996,1.712993,-0.550139,...,1.613138,0.004014,1.682438,1.178400,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
998,0.285720,-0.990183,0.034152,0.720506,0.123499,-1.095692,-1.351071,-1.460029,-1.403212,0.038804,...,-0.328421,0.004033,-0.684241,-0.719407,-1.758640,-0.389139,0.300922,-0.784032,-0.47219,NaN


In [11]:
df1['intOwMemberSeq'] = df['intOwMemberSeq']
df1

C:\Users\yeons\AppData\Local\Temp\ipykernel_19256\3184527928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['intOwMemberSeq'] = df['intOwMemberSeq']


,intSeq,intStatus,bShare,intOwMemberSeq,strUpZone,strUpAreaCode,strUpJAddress,strDwZone,strDwAreaCode,strDwJAddress,...,strDownloadType,intDistance,intDefaultPrice,fltRealWeight,intActApp,intUpDateType,intDwDateType,intCarManageSeq,intMemberSeq,intFeeSend
0,0.285695,-0.990183,0.034152,57037,0.123499,-1.329024,-1.572152,-1.011072,-0.918296,-0.170030,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
1,0.285695,-0.990183,0.034152,57037,0.123499,-1.329024,-1.572075,0.448039,0.307204,1.392233,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
2,0.285695,-0.990183,0.034152,63816,0.123499,-1.277805,-1.481069,0.223560,-0.246875,-1.499758,...,-0.328421,0.004005,-0.684241,1.178400,-1.758640,-0.389139,-0.908531,-0.784032,-0.30799,NaN
3,0.285695,0.966049,0.034152,55238,-1.623514,1.694531,0.493004,-1.460029,-1.465381,-0.099029,...,-0.328421,0.004034,-0.684241,1.356320,-1.416451,-0.389139,0.300922,-0.784032,NaN,NaN
4,0.285695,-0.990183,0.034152,41258,0.123499,-0.733906,-0.851235,0.223560,-0.626104,0.742935,...,-0.328421,0.003997,0.147795,-0.719407,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.285720,0.966049,0.034152,56410,0.795428,-0.006270,0.801138,0.672517,0.996499,0.966380,...,-0.328421,0.004009,-0.684241,1.297013,-1.758640,-0.389139,0.300922,-0.784032,NaN,NaN
996,0.285720,-0.990183,0.034152,39494,0.123499,-0.759922,-0.933094,0.896996,1.684240,-0.565905,...,-0.328421,0.004015,1.711405,1.059787,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
997,0.285720,0.966049,0.034152,53758,0.661042,0.287224,0.938732,0.896996,1.712993,-0.550139,...,1.613138,0.004014,1.682438,1.178400,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
998,0.285720,-0.990183,0.034152,55047,0.123499,-1.095692,-1.351071,-1.460029,-1.403212,0.038804,...,-0.328421,0.004033,-0.684241,-0.719407,-1.758640,-0.389139,0.300922,-0.784032,-0.47219,NaN


In [12]:
df1['intSeq'] = df['intSeq']
df1

C:\Users\yeons\AppData\Local\Temp\ipykernel_19256\997612670.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['intSeq'] = df['intSeq']


,intSeq,intStatus,bShare,intOwMemberSeq,strUpZone,strUpAreaCode,strUpJAddress,strDwZone,strDwAreaCode,strDwJAddress,...,strDownloadType,intDistance,intDefaultPrice,fltRealWeight,intActApp,intUpDateType,intDwDateType,intCarManageSeq,intMemberSeq,intFeeSend
0,92218976,-0.990183,0.034152,57037,0.123499,-1.329024,-1.572152,-1.011072,-0.918296,-0.170030,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
1,92218977,-0.990183,0.034152,57037,0.123499,-1.329024,-1.572075,0.448039,0.307204,1.392233,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
2,92218978,-0.990183,0.034152,63816,0.123499,-1.277805,-1.481069,0.223560,-0.246875,-1.499758,...,-0.328421,0.004005,-0.684241,1.178400,-1.758640,-0.389139,-0.908531,-0.784032,-0.30799,NaN
3,92218979,0.966049,0.034152,55238,-1.623514,1.694531,0.493004,-1.460029,-1.465381,-0.099029,...,-0.328421,0.004034,-0.684241,1.356320,-1.416451,-0.389139,0.300922,-0.784032,NaN,NaN
4,92218980,-0.990183,0.034152,41258,0.123499,-0.733906,-0.851235,0.223560,-0.626104,0.742935,...,-0.328421,0.003997,0.147795,-0.719407,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,92219972,0.966049,0.034152,56410,0.795428,-0.006270,0.801138,0.672517,0.996499,0.966380,...,-0.328421,0.004009,-0.684241,1.297013,-1.758640,-0.389139,0.300922,-0.784032,NaN,NaN
996,92219973,-0.990183,0.034152,39494,0.123499,-0.759922,-0.933094,0.896996,1.684240,-0.565905,...,-0.328421,0.004015,1.711405,1.059787,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
997,92219974,0.966049,0.034152,53758,0.661042,0.287224,0.938732,0.896996,1.712993,-0.550139,...,1.613138,0.004014,1.682438,1.178400,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
998,92219975,-0.990183,0.034152,55047,0.123499,-1.095692,-1.351071,-1.460029,-1.403212,0.038804,...,-0.328421,0.004033,-0.684241,-0.719407,-1.758640,-0.389139,0.300922,-0.784032,-0.47219,NaN


## 유저 만들기 및 특성, 가중치 부여

In [13]:
# 데이터 프레임에서 유저 ID 추출하기
user_id_array = df['intOwMemberSeq'].unique()
user_id_array

array([57037, 63816, 55238, ..., 17168, 48012, 36815])

In [14]:
# 유저 특성 추출하기
# 차량종류, 차량톤수 ,결제 방법, 기사운임,상차종류,하차종류
#user_features = df1.set_index('intSeq')[['strCarType','strCarSize','intPaytypecode','intMemberPrice','intUpDateType','intDwDateType']].groupby('intSeq')
user_features = df1.set_index('intOwMemberSeq')[['strPaytype','strUploadType','strCarType','intMemberPrice','strUpAreaCode','intArrangePrice','intDistance']].groupby('intOwMemberSeq')

In [15]:
# 결측값을 0으로 대체
user_features = user_features.fillna(0)

# 유저 목록 출력
print(user_features)

                strPaytype  strUploadType  strCarType  intMemberPrice  \
intOwMemberSeq                                                          
57037            -0.169079      -0.331450   -1.544589       -0.213766   
57037            -0.169079      -0.331450   -1.544589       -0.277374   
57037            -0.169079      -0.331450   -1.544589       -0.340983   
57037            -0.169079      -0.331450   -1.544589       -0.340983   
57037            -0.169079      -0.331450   -1.544589       -0.277374   
...                    ...            ...         ...             ...   
56410            -0.169079      -0.331450    0.483332       -0.086548   
56410            -0.169079      -0.331450    0.483332       -0.086548   
53758            -0.169079       1.559569    0.483332        0.167887   
55047            -0.169079      -0.331450    0.483332        0.358713   
8586             -0.169079      -0.331450    0.483332       -0.181961   

                strUpAreaCode  intArrangePrice  in

In [16]:
# 각 특성의 가중치 설정
strPaytype_weight = 1
strUploadType_weight  = 1
strCarType_weight  = 1
intMemberPrice_weight = 1
strUpAreaCode_weight = 1
intArrangePrice_weight = 1
intDistance_weight = 1

In [17]:
# 각 특성에 가중치 부여
user_features['strPaytype'] *= strPaytype_weight 
user_features['strUploadType'] *= strUploadType_weight 
user_features['strCarType'] *= strCarType_weight 
user_features['intMemberPrice'] *= intMemberPrice_weight
user_features['strUpAreaCode'] *= strUpAreaCode_weight
user_features['intArrangePrice'] *= intArrangePrice_weight
user_features['intDistance'] *= intDistance_weight

In [18]:
# 유저 목록 출력
print(user_features.head())

                strPaytype  strUploadType  strCarType  intMemberPrice  \
intOwMemberSeq                                                          
57037            -0.169079       -0.33145   -1.544589       -0.213766   
57037            -0.169079       -0.33145   -1.544589       -0.277374   
57037            -0.169079       -0.33145   -1.544589       -0.340983   
57037            -0.169079       -0.33145   -1.544589       -0.340983   
57037            -0.169079       -0.33145   -1.544589       -0.277374   

                strUpAreaCode  intArrangePrice  intDistance  
intOwMemberSeq                                               
57037               -1.329024        -0.177533     0.004006  
57037               -1.329024        -0.177533     0.004006  
57037               -0.891629        -0.177533     0.003995  
57037               -0.891629        -0.177533     0.003995  
57037               -0.891629        -0.177533     0.003995  


## 아이템 만들기 및 특성, 가중치 부여

In [19]:
# 데이터 프레임에서 아이템 ID 추출하기
item_id = df['intSeq'].unique()
item_id

array([92218976, 92218977, 92218978, ...,  1221532,  1221533,  1221534],
      dtype=int64)

In [20]:
# 아이템 특성 추출하기
# 상차지역, 상차위치, 상차지번,하차지역, 하차위치, 하자지번, 화물 형태, 실거리, 화물 실중량 
#item_features = df1.groupby('intSeq')[['strUpZone','strUpAreaCode','strUpJAddress','strDwZone','strDwAreaCode','strDwJAddress','strGoodsType','intDistance','fltRealWeight']]

item_features = df1.groupby('intSeq')[['strGoodsType','fltRealWeight','strDownloadType','strDwAreaCode']]


In [21]:
# 결측값을 0으로 대체
item_features = item_features.fillna(0)

# 아이템 목록 출력
print(item_features.head())

   strGoodsType  fltRealWeight  strDownloadType  strDwAreaCode
0      0.485106      -1.075246        -0.328421      -0.918296
1      0.485106      -1.075246        -0.328421       0.307204
2      0.485106       1.178400        -0.328421      -0.246875
3      0.485106       1.356320        -0.328421      -1.465381
4      0.485106      -0.719407        -0.328421      -0.626104


In [22]:
# 각 특성의 가중치 설정
strGoodsType_weight = 2
fltRealWeight_weight = 1
strDownloadType_weight = 1
strDwAreaCode_weight = 1

In [23]:
# 각 특성에 가중치 부여
item_features['strGoodsType'] *= strGoodsType_weight 
item_features['fltRealWeight'] *= fltRealWeight_weight 
item_features['strDownloadType'] *= strDownloadType_weight
item_features['strDwAreaCode'] *= strDwAreaCode_weight

In [24]:
# 아이템 목록 출력
print(item_features.head())

   strGoodsType  fltRealWeight  strDownloadType  strDwAreaCode
0      0.970212      -1.075246        -0.328421      -0.918296
1      0.970212      -1.075246        -0.328421       0.307204
2      0.970212       1.178400        -0.328421      -0.246875
3      0.970212       1.356320        -0.328421      -1.465381
4      0.970212      -0.719407        -0.328421      -0.626104


In [25]:
user_features.shape

(1000, 7)

In [26]:
item_features.shape

(1000, 4)

# 원본

In [27]:
class AutoencoderRecommender:
    def __init__(self, user_features, item_features, latent_dim=12, encoding_dim=12):
        self.user_features = user_features
        self.item_features = item_features
        self.encoding_dim = encoding_dim
        self.scaler = MinMaxScaler()
        self.user_encoder, self.user_decoder, self.user_autoencoder = self.build_autoencoder()
        self.item_encoder, self.item_decoder, self.item_autoencoder = self.build_autoencoder()

    def build_autoencoder(self):
        input_dim = self.user_features.shape[1]

        # Encoder
        input_layer = Input(shape=(input_dim,))
        encoded = Dense(50, activation='relu')(input_layer)
        encoded = Dense(15, activation='relu')(encoded)
        encoded = Dense(self.encoding_dim, activation='relu')(encoded)

        # Decoder
        decoded = Dense(15, activation='relu')(encoded)
        decoded = Dense(50, activation='relu')(decoded)
        decoded = Dense(input_dim, activation='sigmoid')(decoded)

        # Autoencoder
        autoencoder = Model(inputs=input_layer, outputs=decoded)
        autoencoder.compile(optimizer='adam', loss='mse')

        # Encoder
        encoder = Model(inputs=input_layer, outputs=encoded)

        # Decoder
        decoded_input = Input(shape=(self.encoding_dim,))
        decoded_layer1 = autoencoder.layers[-3](decoded_input)
        decoded_layer2 = autoencoder.layers[-2](decoded_layer1)
        decoded_layer3 = autoencoder.layers[-1](decoded_layer2)
        decoder = Model(inputs=decoded_input, outputs=decoded_layer3)

        return encoder, decoder, autoencoder

    def fit(self, user_epochs=10, item_epochs=10, batch_size=32):
        user_features = self.user_features.values
        item_features = self.item_features.values
        user_features = self.scaler.fit_transform(user_features)
        item_features = self.scaler.fit_transform(item_features)

        self.user_autoencoder.fit(user_features, user_features,
                                  epochs=user_epochs,
                                  batch_size=batch_size,
                                  shuffle=True)
        self.item_autoencoder.fit(item_features, item_features,
                                  epochs=item_epochs,
                                  batch_size=batch_size,
                                  shuffle=True)

        # Encode user features and item features
        self.user_features_encoded = self.user_encoder.predict(user_features)
        self.item_features_encoded = self.item_encoder.predict(item_features)

    def recommend_items(self, user_id, n_items=10):
        # 사용자 특성
        user_features = self.user_features.loc[user_id].values
        user_features = self.scaler.transform([user_features])

        # 특정 유저 인코딩 벡터 
        user_encoded = self.user_encoder.predict(user_features)
        
        # 전체 유저 인코딩 벡터
        user_features_encoded = self.user_encoder.predict(self.user_features.values)
        
        # 특정 아이템 인코딩 벡터
        item_encoded = self.item_encoder.predict(item_features)
        
        # 전체 아이템 인코딩 벡터
        item_features_encoded = self.item_encoder.predict(self.item_features.values)
        
        # 유저간 유사도 계산
        user_similarities = cosine_similarity(user_encoded,user_features_encoded)

        # 가장 유사한 유저들 선정 - 같은 유저는 제외 
        similar_users = np.argsort(-user_similarities[0])[1:]  

        # 유사한 유저들의 아이템 벡터
        
        # 1번 유사한 유저들의 특성을 가져와서 인코딩된 아이템 벡터 (유저 특성 고려 o )
        similar_users_item_features = self.item_encoder.predict(self.user_features.values[similar_users])
        
        # 2번 직접 유사한 유저들의 아이템 벡터를 선택 (유저 특성 고려 x)
        # similar_users_item_features = self.item_features_encoded[similar_users]
        
        # 아이템 간의 유사도 계산하기 
        item_similarities = cosine_similarity(item_encoded, similar_users_item_features)

        # 유사한 아이템 선정하기
        item_indices = np.argsort(-item_similarities[0])[:n_items]

        # 평점 시스템 계산하기 
        predicted_ratings = np.dot(item_similarities[0][item_indices], self.item_features_encoded[item_indices])
        
        # 최저점 최대점 설정하기 
        min_rating = 0
        max_rating = 5
        predicted_ratings = (predicted_ratings - np.min(predicted_ratings)) / (np.max(predicted_ratings) - np.min(predicted_ratings))
        predicted_ratings = predicted_ratings * (max_rating - min_rating) + min_rating

        # 추천 아이템 선정하기
        recommended_items = self.item_features.iloc[item_indices].index.values.tolist()
         
        # 유사도와 추천 아이템을 함께 반환
        item_similarities = item_similarities[0][item_indices].tolist()
        recommended_items_with_similarities = list(zip(recommended_items, item_similarities))

        return recommended_items_with_similarities, predicted_ratings


# 수정본

In [28]:
class AutoencoderRecommender:
    def __init__(self, user_features, item_features, latent_dim=7, encoding_dim=4):
        self.user_features = user_features
        self.item_features = item_features
        self.encoding_dim = encoding_dim
        self.scaler = MinMaxScaler()
        self.user_encoder, self.user_autoencoder = self.build_autoencoder(self.user_features.shape[1])
        self.item_encoder, self.item_autoencoder = self.build_autoencoder(self.item_features.shape[1])

    def build_autoencoder(self, input_dim):
        # Encoder
        input_layer = Input(shape=(input_dim,))
        encoded = Dense(50, activation='relu')(input_layer)
        encoded = Dense(15, activation='relu')(encoded)
        encoded = Dense(self.encoding_dim, activation='relu')(encoded)

        # Decoder
        decoded = Dense(15, activation='relu')(encoded)
        decoded = Dense(50, activation='relu')(decoded)
        decoded = Dense(input_dim, activation='sigmoid')(decoded)

        # Autoencoder
        autoencoder = Model(inputs=input_layer, outputs=decoded)
        autoencoder.compile(optimizer='adam', loss='mse')

        # Encoder
        encoder = Model(inputs=input_layer, outputs=encoded)

        return encoder, autoencoder

    def fit(self, user_epochs=10, item_epochs=10, batch_size=32):
        user_features = self.user_features.values
        item_features = self.item_features.values
        user_features = self.scaler.fit_transform(user_features)
        item_features = self.scaler.fit_transform(item_features)

        self.user_autoencoder.fit(user_features, user_features,
                                  epochs=user_epochs,
                                  batch_size=batch_size,
                                  shuffle=True)
        self.item_autoencoder.fit(item_features, item_features,
                                  epochs=item_epochs,
                                  batch_size=batch_size,
                                  shuffle=True)

        # Encode user features and item features
        self.user_features_encoded = self.user_encoder.predict(user_features)
        self.item_features_encoded = self.item_encoder.predict(item_features)

    def recommend_items(self, user_id, n_items=10):
        # 사용자 특성
        user_features = self.user_features.loc[user_id].values
        user_features = self.scaler.transform([user_features])

        # 특정 유저 인코딩 벡터
        user_encoded = self.user_encoder.predict(user_features)

        # 전체 유저 인코딩 벡터
        user_features_encoded = self.user_encoder.predict(self.user_features.values)

        # 특정 아이템 인코딩 벡터
        item_encoded = self.item_encoder.predict(self.item_features.values)

        # 전체 아이템 인코딩 벡터
        item_features_encoded = self.item_encoder.predict(self.item_features.values)

        # 유저간 유사도 계산
        user_similarities = cosine_similarity(user_encoded, user_features_encoded)

        # 가장 유사한 유저들 선정 - 같은 유저는 제외
        similar_users = np.argsort(-user_similarities[0])[1:]

        # 유사한 유저들의 아이템 벡터
        similar_users_item_features = self.item_encoder.predict(self.user_features.values[similar_users])

        # 아이템 간의 유사도 계산하기
        item_similarities = cosine_similarity(item_encoded, similar_users_item_features)

        # 유사한 아이템 선정하기
        item_indices = np.argsort(-item_similarities[0])[:n_items]

        # 평점 시스템 계산하기
        predicted_ratings = np.dot(item_similarities[0][item_indices], item_features_encoded[item_indices])

        # 최저점 최대점 설정하기
        min_rating = 0
        max_rating = 5
        predicted_ratings = (predicted_ratings - np.min(predicted_ratings)) / (
                np.max(predicted_ratings) - np.min(predicted_ratings))
        predicted_ratings = predicted_ratings * (max_rating - min_rating) + min_rating

        # 추천 아이템 선정하기
        recommended_items = self.item_features.iloc[item_indices].index.values.tolist()

        # 유사도와 추천 아이템을 함께 반환
        item_similarities = item_similarities[0][item_indices].tolist()
        recommended_items_with_similarities = list(zip(recommended_items, item_similarities))

        return recommended_items_with_similarities, predicted_ratings


In [29]:
# AutoencoderRecommender 객체를 생성한다.
recommender = AutoencoderRecommender(user_features, item_features, latent_dim=7, encoding_dim=4)

# 오토인코더 모델을 학습한다.
recommender.fit(user_epochs=10, item_epochs=10, batch_size=32)


Epoch 1/10
32/32 [==============================] - 1s 1ms/step - loss: 0.0859
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0548
Epoch 3/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0343
Epoch 4/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0262
Epoch 5/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0192
Epoch 6/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0160
Epoch 7/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0131
Epoch 8/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0108
Epoch 9/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0096
Epoch 10/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0090
Epoch 1/10
32/32 [==============================] - 1s 1ms/step - loss: 0.1055
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0484
Epoch 3/10
32/32 [==============================] -

In [30]:
def print_recommendations(recommender, user_id, n_items=10):
    recommendations, predicted_ratings = recommender.recommend_items(user_id, n_items)

    print(f"Recommendations for User {user_id}:")
    for item, similarity in recommendations:
        print(f"- Item: {item}, Similarity: {similarity:.4f}")

    print("\nPredicted Ratings:")
    for item, rating in zip(recommendations, predicted_ratings):
        print(f"- Item: {item[0]}, Predicted Rating: {rating:.4f}")


In [31]:
# 특정 유저에게 아이템을 추천한다.
user_id = 57037.0

user_id = input("Enter the user ID: ")
user_id = float(user_id)

n_items = 10

recommendations, predicted_ratings = recommender.recommend_items(user_id, n_items)

print(f"Recommendations for User {user_id}:")
for item, similarity in recommendations:
    print(f"- Item: {item}, Similarity: {similarity:.4f}")

print("\nPredicted Ratings:")
for rating in predicted_ratings.flatten():
    print(f"- Predicted Rating: {rating:.4f}")
for rating in predicted_ratings:
    print(f"- Predicted Rating: {rating:.4f}")

Enter the user ID: 57037


ValueError: Found array with dim 3. Estimator expected <= 2.